In [4]:
import numpy as np
import pandas as pd

# Creating Dummy Dataset for Prototyping
---
Pass if:
- flow_metric is between 9 and 11 µL/min
- pressure drop is between 85 and 115 Pascals

In [7]:
data = pd.read_csv('datasets/AI_Simulation_Microfluidic_Dataset.csv')

In [8]:
print(data)

    model_id  layer_thickness  exposure_time  printer_temp  resin_type  \
0          0               50       6.399748     34.430564  ShapeInk55   
1          1              100       7.209848     33.125870  ShapeInk55   
2          2              100       4.228079     34.328546  ShapeInk55   
3          3              100       3.827319     27.943649  ShapeInk55   
4          4               50       7.277835     29.928785  ShapeInk55   
..       ...              ...            ...           ...         ...   
95        95               50       5.800096     26.131741  ShapeInk55   
96        96               50       6.659466     33.764657  ShapeInk55   
97        97              100       6.183770     31.144713  ShapeInk55   
98        98              100       6.126499     29.908209  ShapeInk55   
99        99              100       5.095084     27.039747  ShapeInk55   

    resin_temp  resin_age  print_res_X  print_res_Y  layer_offset  \
0    21.592094        0.0    57.643310    